In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import matplotlib.pyplot as plt
import torchvision

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Argument Declare

In [2]:
DATASET_ROOT = "/home/austin/DataSet/ncsist_dataset/ncsist"
DATASET_NAME = "ncsisT"
cfg = subt
BASE_NET = "./weights/vgg16_reducedfc.pth"
DATA_DETECTION = SUBTDetection
BATCH_SIZE = 2
PRETRAINED_MODEL = None
PRETRAINED_ITER = 0
SAVE_MODEL_ITER = 500
START_ITER = 0
NUM_WORKERS = 4
CUDA = True
LR = 1e-3
MOMENTUM = 0.4
WEIGHT_DECAY = 5e-4
GAMMA = 0.1
VISDOM = False
SAVE_FOLDER = "./weights/" + DATASET_NAME + "/"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
print('done')
print(DATA_DETECTION)

done
<class 'data.subt_artifact.SUBTDetection'>


In [3]:
if torch.cuda.is_available():
    if not CUDA:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

## Define function

In [4]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# Initial model weights & bias
def xavier(param):
    init.xavier_uniform(param)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

# Adjust learning rate during training
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = LR * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("Change learning rate to: ", lr)

## Create dataset

In [5]:
dataset = DATA_DETECTION(root=DATASET_ROOT, image_sets=['train'],transform=SSDAugmentation(cfg['min_dim'], MEANS))

classes = dataset.target_transform.class_to_ind
print("Class to index: \n", classes)
classes = sorted(classes.items(), key=lambda kv: kv[1])
label = []
for i in classes:
    label.append(i[0])
label.append('None')
print(label)
true_label = ['missle', 'backpack', 'blueline', 'drill', 'can']
print(dataset.target_transform)

Class to index: 
 {'missle': 0, 'backpack': 1, 'blueline': 2, 'drill': 3, 'can': 4}
['missle', 'backpack', 'blueline', 'drill', 'can', 'None']


## Create model

In [6]:
# Delcare SSD Network
#ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
ssd_net = build_ssd('train', cfg['min_dim'], 6)
net = ssd_net
if CUDA:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
    
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
    ssd_net.load_weights(SAVE_FOLDER + PRETRAINED_MODEL)
else:
    print('Initializing weights...')
    vgg_weights = torch.load(BASE_NET) # load vgg pretrained model
    ssd_net.vgg.load_state_dict(vgg_weights)
    ssd_net.extras.apply(weights_init) # Initial SSD model weights & bias
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM,
                weight_decay=WEIGHT_DECAY)
print(cfg['min_dim'])
#criterion = MultiBoxLoss(BATCH_SIZE ,cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                #False, CUDA)
criterion = MultiBoxLoss(BATCH_SIZE ,6, 0.5, True, 0, True, 3, 0.5,False, CUDA)

Initializing weights...
300


/home/austin/SSD-SubT/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/austin/SSD-SubT/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


## Training

### Initialize

In [7]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')
epoch_size = len(dataset) // BATCH_SIZE
print('Training SSD on:', DATASET_NAME)

data_loader = data.DataLoader(dataset, BATCH_SIZE,
                                num_workers=NUM_WORKERS,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
batch_iterator = iter(data_loader)

Loading the dataset...
Training SSD on: ncsisT


/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

### Start training

In [8]:
step_index = 0
for iteration in range(START_ITER, cfg['max_iter']):
    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, GAMMA, step_index)
    
    # make sure data iter not out of range
    try:
        images, targets = next(batch_iterator)
        #print(targets[0][0][4].item(), label[int(targets[0][0][4].item())])
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    if CUDA:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    
    # Forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()
    
    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(PRETRAINED_ITER + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')
    
    if iteration != 0 and iteration % SAVE_MODEL_ITER == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                       repr(PRETRAINED_ITER + iteration) + '.pth')
# Save final model
torch.save(ssd_net.state_dict(),
            SAVE_FOLDER + DATASET_NAME + '.pth')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/home/austin/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 1.7200 sec.
iter 0 || Loss: 28.6628 ||timer: 0.0752 sec.
iter 10 || Loss: 19.1131 ||timer: 0.0747 sec.
iter 20 || Loss: 19.0386 ||timer: 0.0765 sec.
iter 30 || Loss: 13.4547 ||timer: 0.0748 sec.
iter 40 || Loss: 15.8399 ||timer: 0.0752 sec.
iter 50 || Loss: 37.9447 ||timer: 0.0748 sec.
iter 60 || Loss: 21.6317 ||timer: 0.0751 sec.
iter 70 || Loss: 13.3289 ||timer: 0.0755 sec.
iter 80 || Loss: 25.9155 ||timer: 0.0751 sec.
iter 90 || Loss: 16.0891 ||timer: 0.0746 sec.
iter 100 || Loss: 41.9437 ||timer: 0.0750 sec.
iter 110 || Loss: 14.9400 ||timer: 0.0754 sec.
iter 120 || Loss: 18.1226 ||timer: 0.0756 sec.
iter 130 || Loss: 25.3969 ||timer: 0.0753 sec.
iter 140 || Loss: 14.3746 ||timer: 0.0753 sec.
iter 150 || Loss: 14.1711 ||timer: 0.0752 sec.
iter 160 || Loss: 14.2845 ||timer: 0.0772 sec.
iter 170 || Loss: 9.3257 ||timer: 0.0757 sec.
iter 180 || Loss: 8.8763 ||timer: 0.0759 sec.
iter 190 || Loss: 8.2257 ||timer: 0.0753 sec.
iter 200 || Loss: 10.4218 ||timer: 0.0749 sec.
iter 210

/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

timer: 0.0758 sec.
iter 1670 || Loss: 7.6885 ||timer: 0.0761 sec.
iter 1680 || Loss: 4.5052 ||timer: 0.0780 sec.
iter 1690 || Loss: 6.0198 ||timer: 0.0781 sec.
iter 1700 || Loss: 5.2380 ||timer: 0.0767 sec.
iter 1710 || Loss: 5.3748 ||timer: 0.0761 sec.
iter 1720 || Loss: 5.1114 ||timer: 0.0796 sec.
iter 1730 || Loss: 8.1485 ||timer: 0.0761 sec.
iter 1740 || Loss: 5.6234 ||timer: 0.0774 sec.
iter 1750 || Loss: 6.3943 ||timer: 0.0759 sec.
iter 1760 || Loss: 5.6240 ||timer: 0.0756 sec.
iter 1770 || Loss: 6.2017 ||timer: 0.0755 sec.
iter 1780 || Loss: 5.3456 ||timer: 0.0772 sec.
iter 1790 || Loss: 5.2520 ||timer: 0.0766 sec.
iter 1800 || Loss: 9.8484 ||timer: 0.0758 sec.
iter 1810 || Loss: 6.4517 ||timer: 0.0766 sec.
iter 1820 || Loss: 3.7072 ||timer: 0.0788 sec.
iter 1830 || Loss: 9.3874 ||timer: 0.0761 sec.
iter 1840 || Loss: 7.5446 ||timer: 0.0771 sec.
iter 1850 || Loss: 5.0596 ||timer: 0.0757 sec.
iter 1860 || Loss: 4.5784 ||timer: 0.0798 sec.
iter 1870 || Loss: 4.3188 ||timer: 0.0755

/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

...

timer: 0.0771 sec.
iter 11630 || Loss: 2.5561 ||timer: 0.0801 sec.
iter 11640 || Loss: 1.5226 ||timer: 0.0763 sec.
iter 11650 || Loss: 2.6904 ||timer: 0.0799 sec.
iter 11660 || Loss: 3.1283 ||timer: 0.0771 sec.
iter 11670 || Loss: 1.0787 ||timer: 0.0784 sec.
iter 11680 || Loss: 2.1166 ||timer: 0.0761 sec.
iter 11690 || Loss: 1.1030 ||timer: 0.0804 sec.
iter 11700 || Loss: 1.7350 ||timer: 0.0809 sec.
iter 11710 || Loss: 3.1266 ||timer: 0.0774 sec.
iter 11720 || Loss: 3.5543 ||timer: 0.0798 sec.
iter 11730 || Loss: 1.5957 ||timer: 0.0771 sec.
iter 11740 || Loss: 1.9276 ||timer: 0.0788 sec.
iter 11750 || Loss: 1.7547 ||timer: 0.0791 sec.
iter 11760 || Loss: 2.0279 ||timer: 0.0798 sec.
iter 11770 || Loss: 2.9703 ||timer: 0.0770 sec.
iter 11780 || Loss: 3.1501 ||timer: 0.0803 sec.
iter 11790 || Loss: 1.1881 ||timer: 0.0783 sec.
iter 11800 || Loss: 2.4715 ||timer: 0.0768 sec.
iter 11810 || Loss: 2.4596 ||timer: 0.0759 sec.
iter 11820 || Loss: 2.4036 ||timer: 0.0784 sec.
iter 11830 || Loss: 4

/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

timer: 0.0757 sec.
iter 13290 || Loss: 3.9264 ||timer: 0.0757 sec.
iter 13300 || Loss: 3.7026 ||timer: 0.0766 sec.
iter 13310 || Loss: 2.0460 ||timer: 0.0775 sec.
iter 13320 || Loss: 3.5959 ||timer: 0.0792 sec.
iter 13330 || Loss: 1.7089 ||timer: 0.0763 sec.
iter 13340 || Loss: 2.0348 ||timer: 0.0815 sec.
iter 13350 || Loss: 2.4017 ||timer: 0.0774 sec.
iter 13360 || Loss: 2.7156 ||timer: 0.0764 sec.
iter 13370 || Loss: 2.6018 ||timer: 0.0784 sec.
iter 13380 || Loss: 2.0541 ||timer: 0.0777 sec.
iter 13390 || Loss: 2.0404 ||timer: 0.0768 sec.
iter 13400 || Loss: 2.6768 ||timer: 0.0808 sec.
iter 13410 || Loss: 2.1300 ||timer: 0.0788 sec.
iter 13420 || Loss: 4.4706 ||timer: 0.0765 sec.
iter 13430 || Loss: 3.0868 ||timer: 0.0793 sec.
iter 13440 || Loss: 3.3256 ||timer: 0.0804 sec.
iter 13450 || Loss: 1.9871 ||timer: 0.0813 sec.
iter 13460 || Loss: 1.9917 ||timer: 0.0819 sec.
iter 13470 || Loss: 1.6248 ||timer: 0.0802 sec.
iter 13480 || Loss: 3.6341 ||timer: 0.0762 sec.
iter 13490 || Loss: 3

/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/austin/SSD-SubT/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

timer: 0.0772 sec.
iter 14950 || Loss: 4.5401 ||timer: 0.0786 sec.
iter 14960 || Loss: 2.0144 ||timer: 0.0801 sec.
iter 14970 || Loss: 1.9941 ||timer: 0.0758 sec.
iter 14980 || Loss: 4.9978 ||timer: 0.0788 sec.
iter 14990 || Loss: 5.1595 ||